# **CNN**

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # "0" o "1"

In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found, using CPU.")


2025-10-27 12:51:08.312258: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 12:51:08.367736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-27 12:51:09.587955: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
from utils import *

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import KeyedVectors

In [4]:
def prepare_data(datasets, max_words=20000, max_len=300):
    """
    Tokenization and label encoding for multiple datasets.

    Args:
        datasets (dict): A dictionary where keys are dataset names and values are pandas DataFrames with 'texts' and 'labels' columns.
        max_words (int): Maximum number of words to keep in the tokenizer vocabulary.
        max_len (int): Maximum length of sequences after padding/truncating.

    Returns:
        processed_datasets (dict): A dictionary with the same keys as input, where each value is another dict with 'train', 'val', 'test' splits containing (X, y) tuples.
        tokenizer (Tokenizer): Fitted Keras Tokenizer.
        encoder (LabelEncoder): Fitted sklearn LabelEncoder.
    """
    # Unisci tutti i testi di tutti i dataset
    all_texts = []
    for df in datasets.values():
        all_texts.extend(df["texts"].astype(str).tolist())

    # Tokenizer
    tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>") # tokenizer with OOV token
    tokenizer.fit_on_texts(all_texts) # create vocabulary {word: index}

    # LabelEncoder globale
    all_labels = np.concatenate([df["labels"].values for df in datasets.values()])
    encoder = LabelEncoder().fit(all_labels)

    # Applica tokenizzazione e encoding a ogni dataset
    processed_datasets = {}
    for name, df in datasets.items():
        seq = tokenizer.texts_to_sequences(df["texts"].astype(str).tolist()) # convert texts to sequences of integers
        X = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post') # padd/truncate sequences to max_len
        y = encoder.transform(df["labels"].values) # encode labels to integers

        # Split train/val/test
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

        processed_datasets[name] = {
            "train": (X_train, y_train),
            "val": (X_val, y_val),
            "test": (X_test, y_test)
        }

    return processed_datasets, tokenizer, encoder

In [5]:
def load_word2vec(tokenizer, max_words=20000, embedding_dim=300):
    """
    Load pre-trained Word2Vec embeddings and create embedding matrix

    Args:
        tokenizer: Keras Tokenizer object with fitted vocabulary
        max_words: maximum number of words to consider from tokenizer
        embedding_dim: dimension of Word2Vec embeddings

    Returns:
        embedding_matrix: numpy array of shape (num_words, embedding_dim)
        num_words: actual number of words considered
    """
    
    print("Loading pre-trained Word2Vec model (may take time)...")
    w2v_path = "../Word2Vec_GoogleNews300/word2vec-google-news-300.model"
    w2v_model = KeyedVectors.load(w2v_path, mmap='r') # load model with memory mapping (mmap='r' for only reading) 

    word_index = tokenizer.word_index               # vocabulary from tokenizer {word: index}
    num_words = min(max_words, len(word_index) + 1) # number of words to consider (max_words or vocab size)

    # create embedding matrix: each row corresponds to a word index from tokenizer, each column to an embedding dimension
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for word, i in word_index.items():
        if i >= max_words: # skip because we only consider top max_words
            continue
        if word in w2v_model: # if word has a pre-trained embedding
            embedding_matrix[i] = w2v_model[word]

    return embedding_matrix, num_words

In [6]:
# -----------------------
# Model building function
# -----------------------

def build_model(embedding_matrix, num_words, max_len=300, embedding_dim=300,
                filter_size=3, num_filters=64, dropout=0.4,
                hidden_units=32, learning_rate=1e-4):
    """
    Build and compile the CNN model.

    Args:
        embedding_matrix: Pre-trained embedding matrix
        num_words: Number of words in the vocabulary
        max_len: Maximum length of input sequences
        embedding_dim: Dimension of word embeddings
        filter_size: Size of the convolutional filters
        num_filters: Number of convolutional filters
        dropout: Dropout rate
        hidden_units: Number of units in the dense hidden layer
        learning_rate: Learning rate for the Adam optimizer

    Returns:
        model: Compiled Keras model
    """
    model = Sequential([
        Embedding(num_words,
                  embedding_dim,
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=True),
        Dropout(dropout),
        Conv1D(num_filters, filter_size, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(hidden_units, activation='relu'),
        Dropout(dropout),
        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## VERSION 1: Dataset (Simple)

In [7]:
dataset_df = data_loading() # load datasets

for name, df in dataset_df.items():
    print(f"Dataset: {name}, Number of samples: {len(df)}")

Dataset: Celebrity, Number of samples: 500
Dataset: CIDII, Number of samples: 722
Dataset: FaKES, Number of samples: 842
Dataset: FakeVsSatire, Number of samples: 486
Dataset: Horne, Number of samples: 326
Dataset: Infodemic, Number of samples: 10559
Dataset: ISOT, Number of samples: 44271
Dataset: Kaggle_clement, Number of samples: 39105
Dataset: Kaggle_meg, Number of samples: 12845
Dataset: LIAR_PLUS, Number of samples: 12784
Dataset: Politifact, Number of samples: 504
Dataset: Unipi_NDF, Number of samples: 554


/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT


In [8]:
# --------------------------------
# Fine-tuning on multiple datasets
# --------------------------------

# preprocessing on all datasets
datasets, tokenizer, encoder = prepare_data(dataset_df)
embedding_matrix, num_words = load_word2vec(tokenizer)

# initialize model
model = build_model(embedding_matrix, num_words)

results = {}

# sequential training
for i, (name, data) in enumerate(datasets.items()):
    print(f"\n=== Phase {i+1}: Training/Fine-tuning on {name} ===")
    
    X_train, y_train = data["train"]
    X_val, y_val = data["val"]
    X_test, y_test = data["test"]

    # early stopping
    es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

    # fine-tune on train + val
    model.fit(
        np.concatenate([X_train, X_val]),
        np.concatenate([y_train, y_val]),
        epochs=10,
        batch_size=8,
        validation_data=(X_val, y_val),
        callbacks=[es],
        verbose=1
    )

    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    print(f"Classification Report after {name}:")
    print(classification_report(y_test, y_pred))
    print(f"Confusion Matrix after {name}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\nWeighted F1-score after {name}:", f1_score(y_test, y_pred, average="weighted"))


    # evaluation on all datasets
    print("\n--- Evaluation on all datasets ---")
    results[name] = {}
    for test_name, test_data in datasets.items(): # for each dataset
        X_te, y_te = test_data["test"]
        preds = model.predict(X_te)
        preds = (preds > 0.5).astype(int)
        f1 = f1_score(y_te, preds, average="weighted")
        results[name][test_name] = f1
        print(f"Evaluation on {test_name}: Weighted F1 = {f1:.4f}")


Loading pre-trained Word2Vec model (may take time)...


/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1761565947.018184 3391636 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 29192 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0



=== Phase 1: Training/Fine-tuning on Celebrity ===
Epoch 1/10


2025-10-27 12:52:28.794851: I external/local_xla/xla/service/service.cc:163] XLA service 0x793c6401ca30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-27 12:52:28.794882: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2025-10-27 12:52:28.839535: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-27 12:52:29.000613: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


35/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4448 - loss: 0.7871

I0000 00:00:1761565951.020361 3392366 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.4725 - loss: 0.7606 - val_accuracy: 0.5400 - val_loss: 0.6861
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5175 - loss: 0.7261 - val_accuracy: 0.6400 - val_loss: 0.6772
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5225 - loss: 0.7062 - val_accuracy: 0.6500 - val_loss: 0.6713
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5325 - loss: 0.7147 - val_accuracy: 0.6900 - val_loss: 0.6626
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5200 - loss: 0.6928 - val_accuracy: 0.6900 - val_loss: 0.6554
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5750 - loss: 0.6742 - val_accuracy: 0.7800 - val_loss: 0.6434
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5500 - loss: 0.6863 - val_accuracy: 0.8800 - val_loss: 0.6335
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5800 - loss: 0.6849 - val_accuracy: 0.8800 - val_loss: 0.6249
Ep

In [9]:
# ---------------
# Results summary
# ---------------

print("\n=== Results Summary ===")
for name, res in results.items():
    print(f"\nResults after training on {name}:")
    for test_name, f1 in res.items():
        print(f"  Test on {test_name}: Weighted F1 = {f1:.4f}")


=== Results Summary ===

Results after training on Celebrity:
  Test on Celebrity: Weighted F1 = 0.6692
  Test on CIDII: Weighted F1 = 0.4962
  Test on FaKES: Weighted F1 = 0.4911
  Test on FakeVsSatire: Weighted F1 = 0.4344
  Test on Horne: Weighted F1 = 0.4007
  Test on Infodemic: Weighted F1 = 0.5141
  Test on ISOT: Weighted F1 = 0.5222
  Test on Kaggle_clement: Weighted F1 = 0.5361
  Test on Kaggle_meg: Weighted F1 = 0.7685
  Test on LIAR_PLUS: Weighted F1 = 0.5265
  Test on Politifact: Weighted F1 = 0.6725
  Test on Unipi_NDF: Weighted F1 = 0.5001

Results after training on CIDII:
  Test on Celebrity: Weighted F1 = 0.3967
  Test on CIDII: Weighted F1 = 0.8416
  Test on FaKES: Weighted F1 = 0.4065
  Test on FakeVsSatire: Weighted F1 = 0.4650
  Test on Horne: Weighted F1 = 0.5609
  Test on Infodemic: Weighted F1 = 0.4782
  Test on ISOT: Weighted F1 = 0.4335
  Test on Kaggle_clement: Weighted F1 = 0.4970
  Test on Kaggle_meg: Weighted F1 = 0.9155
  Test on LIAR_PLUS: Weighted F1 = 0

## VERSION 2: Dataset by Topic

In [ ]:
dataset_df = data_by_topic()

for topic, df in dataset_df.items():
    print(f"Topic: {topic}, Number of samples: {len(df)}")

/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT


Topic: politics, Number of samples: 97476
Topic: general, Number of samples: 12845
Topic: covid, Number of samples: 10559
Topic: syria, Number of samples: 842
Topic: islam, Number of samples: 722
Topic: notredame, Number of samples: 554
Topic: gossip, Number of samples: 500


In [ ]:
# -------------------------------
# Fine-tuning on Dataset by Topic
# -------------------------------

# preprocessing on all datasets
datasets, tokenizer, encoder = prepare_data(dataset_df)
embedding_matrix, num_words = load_word2vec(tokenizer)

# initialize model
model = build_model(embedding_matrix, num_words)

results = {}

# sequential training
for i, (topic, data) in enumerate(datasets.items()):
    print(f"\n=== Phase {i+1}: Training/Fine-tuning on topic: {topic} ===")

    X_train, y_train = data["train"]
    X_val, y_val = data["val"]
    X_test, y_test = data["test"]

    # early stopping
    es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

    # fine-tune on train + val
    model.fit(
        np.concatenate([X_train, X_val]),
        np.concatenate([y_train, y_val]),
        epochs=10,
        batch_size=8,
        validation_data=(X_val, y_val),
        callbacks=[es],
        verbose=1
    )

    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    print(f"Classification Report after topic {topic}:")
    print(classification_report(y_test, y_pred))
    print(f"Confusion Matrix after topic {topic}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\nWeighted F1-score after topic {topic}:", f1_score(y_test, y_pred, average="weighted"))


    # evaluation on all topics
    print("\n--- Evaluation on all topics ---")
    results[topic] = {}
    for test_topic, test_data in datasets.items(): # for each topic
        X_te, y_te = test_data["test"]
        preds = model.predict(X_te)
        preds = (preds > 0.5).astype(int)
        f1 = f1_score(y_te, preds, average="weighted")
        results[topic][test_topic] = f1
        print(f"Evaluation on topic {test_topic}: Weighted F1 = {f1:.4f}")


=== Phase 1: Training/Fine-tuning on topic: politics ===
Classification Report after topic politics:
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93     10094
         1.0       0.93      0.93      0.93      9402

    accuracy                           0.93     19496
   macro avg       0.93      0.93      0.93     19496
weighted avg       0.93      0.93      0.93     19496

Confusion Matrix after topic politics:
[[9416  678]
 [ 643 8759]]

Weighted F1-score after topic politics: 0.9322466151502965

--- Evaluation on all topics ---
Evaluation on topic politics: Weighted F1 = 0.9322
Evaluation on topic general: Weighted F1 = 0.2212
Evaluation on topic covid: Weighted F1 = 0.3626
Evaluation on topic syria: Weighted F1 = 0.4746
Evaluation on topic islam: Weighted F1 = 0.4866
Evaluation on topic notredame: Weighted F1 = 0.2855
Evaluation on topic gossip: Weighted F1 = 0.3711

=== Phase 2: Training/Fine-tuning on topic: general ===
Classifi

In [ ]:
# ---------------
# Results summary
# ---------------

print("\n=== Results Summary ===")
for topic, res in results.items():
    print(f"\nResults after training on topic {topic}:")
    for test_topic, f1 in res.items():
        print(f"  Test on topic {test_topic}: Weighted F1 = {f1:.4f}")


=== Results Summary ===

Results after training on topic politics:
  Test on topic politics: Weighted F1 = 0.9322
  Test on topic general: Weighted F1 = 0.2212
  Test on topic covid: Weighted F1 = 0.3626
  Test on topic syria: Weighted F1 = 0.4746
  Test on topic islam: Weighted F1 = 0.4866
  Test on topic notredame: Weighted F1 = 0.2855
  Test on topic gossip: Weighted F1 = 0.3711

Results after training on topic general:
  Test on topic politics: Weighted F1 = 0.3532
  Test on topic general: Weighted F1 = 0.9690
  Test on topic covid: Weighted F1 = 0.3600
  Test on topic syria: Weighted F1 = 0.3633
  Test on topic islam: Weighted F1 = 0.4333
  Test on topic notredame: Weighted F1 = 0.4654
  Test on topic gossip: Weighted F1 = 0.3333

Results after training on topic covid:
  Test on topic politics: Weighted F1 = 0.3696
  Test on topic general: Weighted F1 = 0.1997
  Test on topic covid: Weighted F1 = 0.9399
  Test on topic syria: Weighted F1 = 0.3363
  Test on topic islam: Weighted F

## VERSION 3: Dataset by Date

In [ ]:
dataset_df = data_by_date()

for date, df in dataset_df.items():
    print(f"Date: {date}, Number of samples: {len(df)}")

/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT


Date: 2011-2013, Number of samples: 55
Date: 2014, Number of samples: 114
Date: 2015, Number of samples: 84
Date: 2016, Number of samples: 49687
Date: 2017, Number of samples: 16657
Date: 2020, Number of samples: 10559


In [ ]:
# ------------------------------
# Fine-tuning on Dataset by Date
# ------------------------------

# preprocessing on all datasets
datasets, tokenizer, encoder = prepare_data(dataset_df)
embedding_matrix, num_words = load_word2vec(tokenizer)

# initialize model
model = build_model(embedding_matrix, num_words)

results = {}

# sequential training
for i, (date, data) in enumerate(datasets.items()):
    print(f"\n=== Phase {i+1}: Training/Fine-tuning on date: {date} ===")
    
    X_train, y_train = data["train"]
    X_val, y_val = data["val"]
    X_test, y_test = data["test"]

    # early stopping
    es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

    # fine-tune on train + val
    model.fit(
        np.concatenate([X_train, X_val]),
        np.concatenate([y_train, y_val]),
        epochs=10,
        batch_size=8,
        validation_data=(X_val, y_val),
        callbacks=[es],
        verbose=1
    )

    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    print(f"Classification Report after date {date}:")
    print(classification_report(y_test, y_pred))
    print(f"Confusion Matrix after date {date}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\nWeighted F1-score after date {date}:", f1_score(y_test, y_pred, average="weighted"))


    # evaluation on all dates
    print("\n--- Evaluation on all dates ---")
    results[date] = {}
    for test_date, test_data in datasets.items(): # for each date
        X_te, y_te = test_data["test"]
        preds = model.predict(X_te)
        preds = (preds > 0.5).astype(int)
        f1 = f1_score(y_te, preds, average="weighted")
        results[date][test_date] = f1
        print(f"Evaluation on {test_date}: Weighted F1 = {f1:.4f}")
    


=== Phase 1: Training/Fine-tuning on date: 2011-2013 ===
Classification Report after date 2011-2013:
              precision    recall  f1-score   support

         0.0       1.00      0.20      0.33         5
         1.0       0.60      1.00      0.75         6

    accuracy                           0.64        11
   macro avg       0.80      0.60      0.54        11
weighted avg       0.78      0.64      0.56        11

Confusion Matrix after date 2011-2013:
[[1 4]
 [0 6]]

Weighted F1-score after date 2011-2013: 0.5606060606060606

--- Evaluation on all dates ---
Evaluation on date 2011-2013: Weighted F1 = 0.5606
Evaluation on date 2014: Weighted F1 = 0.2690
Evaluation on date 2015: Weighted F1 = 0.3665
Evaluation on date 2016: Weighted F1 = 0.3004
Evaluation on date 2017: Weighted F1 = 0.0030
Evaluation on date 2020: Weighted F1 = 0.3070

=== Phase 2: Training/Fine-tuning on date: 2014 ===
Classification Report after date 2014:
              precision    recall  f1-score   suppo

/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

Evaluation on date 2016: Weighted F1 = 0.3174
Evaluation on date 2017: Weighted F1 = 0.0667
Evaluation on date 2020: Weighted F1 = 0.3219

=== Phase 4: Training/Fine-tuning on date: 2016 ===
Classification Report after date 2016:
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      5306
         1.0       0.98      0.98      0.98      4632

    accuracy                           0.98      9938
   macro avg       0.98      0.98      0.98      9938
weighted avg       0.98      0.98      0.98      9938

Confusion Matrix after date 2016:
[[5232   74]
 [  80 4552]]

Weighted F1-score after date 2016: 0.9845032811664566

--- Evaluation on all dates ---
Evaluation on date 2011-2013: Weighted F1 = 0.3961
Evaluation on date 2014: Weighted F1 = 0.4214
Evaluation on date 2015: Weighted F1 = 0.3827
Evaluation on date 2016: Weighted F1 = 0.9845
Evaluation on date 2017: Weighted F1 = 0.9915
Evaluation on date 2020: Weighted F1 = 0.3092

=== Phase 5:

/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

Evaluation on date 2016: Weighted F1 = 0.3719
Evaluation on date 2017: Weighted F1 = 0.9937
Evaluation on date 2020: Weighted F1 = 0.3600

=== Phase 6: Training/Fine-tuning on date: 2020 ===
Classification Report after date 2020:
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      1106
         1.0       0.92      0.93      0.93      1006

    accuracy                           0.93      2112
   macro avg       0.93      0.93      0.93      2112
weighted avg       0.93      0.93      0.93      2112

Confusion Matrix after date 2020:
[[1026   80]
 [  68  938]]

Weighted F1-score after date 2020: 0.9299408611809796

--- Evaluation on all dates ---
Evaluation on date 2011-2013: Weighted F1 = 0.3409
Evaluation on date 2014: Weighted F1 = 0.3991
Evaluation on date 2015: Weighted F1 = 0.3388
Evaluation on date 2016: Weighted F1 = 0.3473
Evaluation on date 2017: Weighted F1 = 0.1014
Evaluation on date 2020: Weighted F1 = 0.9299


In [ ]:
# ---------------
# Results summary
# ---------------

print("\n=== Results Summary ===")
for date, res in results.items():
    print(f"\nResults after training on date {date}:")
    for test_date, f1 in res.items():
        print(f"  Test on date {test_date}: Weighted F1 = {f1:.4f}")


=== Results Summary ===

Results after training on date 2011-2013:
  Test on date 2011-2013: Weighted F1 = 0.5606
  Test on date 2014: Weighted F1 = 0.2690
  Test on date 2015: Weighted F1 = 0.3665
  Test on date 2016: Weighted F1 = 0.3004
  Test on date 2017: Weighted F1 = 0.0030
  Test on date 2020: Weighted F1 = 0.3070

Results after training on date 2014:
  Test on date 2011-2013: Weighted F1 = 0.6303
  Test on date 2014: Weighted F1 = 0.4283
  Test on date 2015: Weighted F1 = 0.4938
  Test on date 2016: Weighted F1 = 0.3604
  Test on date 2017: Weighted F1 = 0.8698
  Test on date 2020: Weighted F1 = 0.3937

Results after training on date 2015:
  Test on date 2011-2013: Weighted F1 = 0.3409
  Test on date 2014: Weighted F1 = 0.3095
  Test on date 2015: Weighted F1 = 0.3665
  Test on date 2016: Weighted F1 = 0.3174
  Test on date 2017: Weighted F1 = 0.0667
  Test on date 2020: Weighted F1 = 0.3219

Results after training on date 2016:
  Test on date 2011-2013: Weighted F1 = 0.3961
